In [48]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [49]:
teams_08 = ['ATL', 'NJN', 'BOS', 'CHA', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND',
            'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOH', 'NYK', 'SEA', 'ORL', 'PHI', 'PHO',
            'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']

teams_09_12 = ['ATL', 'NJN', 'BOS', 'CHA', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND',
               'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOH', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO',
               'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']

teams_13 = ['ATL', 'BRK', 'BOS', 'CHA', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND',
            'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOH', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO',
            'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']

teams_14 = ['ATL', 'BRK', 'BOS', 'CHA', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND',
            'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO',
            'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']

teams_15_17 = ['ATL', 'BRK', 'BOS', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND',
               'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO',
               'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']

seasons = ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']

In [50]:
column_headers = ['game_season', 'date_game', 'game_location', 'opp_id', 'game_result', 'pts', 'opp_pts',
                  'fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'trb',
                  'ast', 'stl', 'blk', 'tov', 'pf', 'opp_fg', 'opp_fga', 'opp_fg_pct', 'opp_fg3', 'opp_ft',
                  'opp_fta', 'opp_ft_pct', 'opp_orb', 'opp_trb', 'opp_ast', 'opp_stl', 'opp_blk', 'opp_tov',
                  'opp_pf']
column_headers_additional = ['team', 'season']

In [51]:
advanced_column_headers = ['off_rtg', 'def_rtg', 'pace', 'fta_per_fga_pct', 'fg3a_per_fga_pct', 'ts_pct', 
                           'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'efg_pct', 'tov_pct', 'orb_pct', 
                           'ft_rate', 'opp_efg_pct', 'opp_tov_pct', 'drb_pct', 'opp_ft_rate']
complete_column_headers = column_headers_additional + column_headers + advanced_column_headers

In [54]:
def scrape(team_list, year):
    gamelogs_internal = []
    for team in team_list:
        team_year = [team, year]
        team_data = []

        url = 'https://www.basketball-reference.com/teams/{}/{}/gamelog/'.format(team, year)
        html = urlopen(url)
        soup = BeautifulSoup(html, 'lxml')
        data_rows = soup.findAll('tr', {'id':re.compile('^tgl_basic')})
        data = [[td.getText() for td in data_rows[i].findAll('td', {'data-stat':column_headers})] 
                for i in range(len(data_rows))]

        urlA = 'https://www.basketball-reference.com/teams/{}/{}/gamelog-advanced'.format(team, year)
        htmlA = urlopen(urlA)
        soupA = BeautifulSoup(htmlA, 'lxml')
        data_rowsA = soupA.findAll('tr', {'id':re.compile('^tgl_advanced')})
        dataA = [[td.getText() for td in data_rowsA[i].findAll('td', {'data-stat':advanced_column_headers})]
                for i in range(len(data_rowsA))]

        for row, rowA in zip(data, dataA):
            new_row = team_year + row + rowA
            team_data.append(new_row)
        gamelogs_internal.append(team_data)
        print('{} {} scraped'.format(team, year))
    return gamelogs_internal


def gamelog_scrape(team_list1, team_list2, team_list3, team_list4, team_list5, year_list):
    gamelogs = []
    for year in year_list:
        if year == '2008':
            game_log_piece = scrape(team_list1, year)
            gamelogs.append(game_log_piece)
        elif year in ['2009', '2010', '2011', '2012']:
            game_log_piece = scrape(team_list2, year)
            gamelogs.append(game_log_piece)
        elif year == '2013':
            game_log_piece = scrape(team_list3, year)
            gamelogs.append(game_log_piece)
        elif year == '2014':
            game_log_piece = scrape(team_list4, year)
            gamelogs.append(game_log_piece)
        elif year in ['2015', '2016', '2017']:
            game_log_piece = scrape(team_list5, year)
            gamelogs.append(game_log_piece)
    return gamelogs

In [56]:
data = gamelog_scrape(team_list1=teams_08, team_list2=teams_09_12, team_list3=teams_13, 
                      team_list4=teams_14, team_list5=teams_15_17, year_list=seasons)

ATL 2008 scraped
NJN 2008 scraped
BOS 2008 scraped
CHA 2008 scraped
CHI 2008 scraped
CLE 2008 scraped
DAL 2008 scraped
DEN 2008 scraped
DET 2008 scraped
GSW 2008 scraped
HOU 2008 scraped
IND 2008 scraped
LAC 2008 scraped
LAL 2008 scraped
MEM 2008 scraped
MIA 2008 scraped
MIL 2008 scraped
MIN 2008 scraped
NOH 2008 scraped
NYK 2008 scraped
SEA 2008 scraped
ORL 2008 scraped
PHI 2008 scraped
PHO 2008 scraped
POR 2008 scraped
SAC 2008 scraped
SAS 2008 scraped
TOR 2008 scraped
UTA 2008 scraped
WAS 2008 scraped
ATL 2009 scraped
NJN 2009 scraped
BOS 2009 scraped
CHA 2009 scraped
CHI 2009 scraped
CLE 2009 scraped
DAL 2009 scraped
DEN 2009 scraped
DET 2009 scraped
GSW 2009 scraped
HOU 2009 scraped
IND 2009 scraped
LAC 2009 scraped
LAL 2009 scraped
MEM 2009 scraped
MIA 2009 scraped
MIL 2009 scraped
MIN 2009 scraped
NOH 2009 scraped
NYK 2009 scraped
OKC 2009 scraped
ORL 2009 scraped
PHI 2009 scraped
PHO 2009 scraped
POR 2009 scraped
SAC 2009 scraped
SAS 2009 scraped
TOR 2009 scraped
UTA 2009 scrap

In [58]:
df = pd.concat([pd.DataFrame(d[i]) for d in data for i in range(30)])
df.columns=complete_column_headers
df = df.reset_index(drop=True)

In [63]:
df.to_csv('gamelogs_advanced.csv')